# CityLearn MAML agent
Name: Shota Takeshima
* before implementing MAML, I understand how multiple baseline agent works and remove unnecessary parts, such as information sharing and PCA state deimentionality reduction.

## Installation: CityLearn

* Temporally, I put CityLearn codes in my private git repository to test it. 
* After some progress is confirmed, I'll push the modified code to our team repository.

In [1]:
!rm -rf ./CityLearn_garage/
!git clone https://github.com/shttksm/CityLearn_garage.git

Cloning into 'CityLearn_garage'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 116 (delta 23), reused 107 (delta 17), pack-reused 0
Receiving objects: 100% (116/116), 11.92 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (23/23), done.


## Confirm the baseline multiple agent, RL_Agents_Coord
* RL_Agents_Coord is in `CityLearn_garage/agent.py`. Each building has itw own SAC agent and this class manages such SAC agents.

In [2]:
!grep RL_Agents_Coord CityLearn_garage/agent.py

class RL_Agents_Coord:


### Simulation environment

In [3]:
# Loading libraries
import sys
sys.path.append("./CityLearn_garage")

from citylearn import CityLearn
from reward_function import reward_function_ma
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from agent import RL_Agents_Coord

import os
import gym
import numpy as np
#from stable_baselines3 import SAC
#from stable_baselines3.sac.policies import MlpPolicy as MlpPolicy_SAC
#from stable_baselines3.common.callbacks import BaseCallback
import matplotlib.pyplot as plt
from pathlib import Path
import time

import pandas as pd
import seaborn as sns

import pickle
import copy

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def get_env(climate_zone):
  # Load environment
  data_path = Path("./CityLearn_garage/data/Climate_Zone_"+str(climate_zone))
  building_attributes = data_path / 'building_attributes.json'
  weather_file = data_path / 'weather_data.csv'
  solar_profile = data_path / 'solar_generation_1kW.csv'
  building_state_actions = './CityLearn_garage/buildings_state_action_space.json'
  building_ids = ["Building_1","Building_2","Building_3","Building_4","Building_5","Building_6","Building_7","Building_8","Building_9"]
  objective_function = ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption']

  # Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
  # Can be obtained using observations_spaces[i].low or .high
  env = CityLearn(data_path, 
                  building_attributes, 
                  weather_file, 
                  solar_profile, 
                  building_ids, 
                  buildings_states_actions = building_state_actions, 
                  cost_function = objective_function, 
                  verbose = 1, 
                  simulation_period=(0,8760-1), 
                  central_agent=False)
  # Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
  building_info = env.get_building_information()  
  observations_spaces, actions_spaces = env.get_state_action_spaces()

  return env, building_ids, building_state_actions, building_info, observations_spaces, actions_spaces

In [6]:
env, building_ids, building_state_actions, building_info, observations_spaces, actions_spaces = get_env(1)

In [7]:
observations_spaces

[Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (25,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32),
 Box(-2.200000047683716, 1058.469970703125, (26,), float32)]

In [8]:
actions_spaces

[Box(-0.3333333432674408, 0.3333333432674408, (2,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (2,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (1,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (1,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (2,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (2,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (2,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (2,), float32),
 Box(-0.3333333432674408, 0.3333333432674408, (2,), float32)]

### Train a instance of RI_Agents_Coord
* First in first, I confirm wether the original multiple agents works correctly.

In [9]:
# Hyperparameters
bs = 256
tau = 0.005
gamma = 0.99
lr = 0.0003
hid = [256,256]

n_episodes = 12

In [10]:
# Instantiating the control agent(s)
agents = RL_Agents_Coord(building_ids, 
                         building_state_actions, 
                         building_info, 
                         observations_spaces, 
                         actions_spaces, 
                         discount = gamma, 
                         batch_size = bs, 
                         replay_buffer_capacity = 1e5, 
                         regression_buffer_capacity = 12*8760, 
                         tau=tau, 
                         lr=lr, 
                         hidden_dim=hid, 
                         start_training=8760*3, 
                         exploration_period = 8760*3+1,  
                         start_regression=8760, 
                         information_sharing = True, 
                         pca_compression = .95, 
                         action_scaling_coef=0.5, 
                         reward_scaling = 5., 
                         update_per_step = 1, 
                         iterations_as = 2)

**You can skip here**. This cord just confirming the original agents.

In [11]:
cost_by_epoch = []
# The number of episodes can be replaces by a stopping criterion (i.e. convergence of the average reward)
start = time.time()
for e in range(n_episodes): 
    is_evaluating = (e > 7) # Evaluate deterministic policy after 7 epochs
    rewards = []
    state = env.reset()
    done = False

    j = 0
    action, coordination_vars = agents.select_action(state, deterministic=is_evaluating)    
    while not done:
        next_state, reward, done, _ = env.step(action)
        action_next, coordination_vars_next = agents.select_action(next_state, deterministic=is_evaluating)
        agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)

        state = next_state
        coordination_vars = coordination_vars_next
        action = action_next

    cost = env.cost()
    cost_by_epoch.append(cost)
    print('Loss -', cost, 'Simulation time (min) -',(time.time()-start)/60.0)

Cumulated reward: -168981466.40687048
Cost score: {'ramping': 1.1905503, '1-load_factor': 1.0473200900968902, 'average_daily_peak': 1.0626521, 'peak_demand': 1.2674795, 'net_electricity_consumption': 1.0554651, 'total': 1.1246934339129326}
Loss - {'ramping': 1.1905503, '1-load_factor': 1.0473200900968902, 'average_daily_peak': 1.0626521, 'peak_demand': 1.2674795, 'net_electricity_consumption': 1.0554651, 'total': 1.1246934339129326} Simulation time (min) - 0.9682521859804789
Cumulated reward: -170647247.9540467
Cost score: {'ramping': 1.184989, '1-load_factor': 1.0372429432671828, 'average_daily_peak': 1.0647068, 'peak_demand': 1.2937548, 'net_electricity_consumption': 1.0558867, 'total': 1.1273160565336855}
Loss - {'ramping': 1.184989, '1-load_factor': 1.0372429432671828, 'average_daily_peak': 1.0647068, 'peak_demand': 1.2937548, 'net_electricity_consumption': 1.0558867, 'total': 1.1273160565336855} Simulation time (min) - 5.1020169695218405
Cumulated reward: -170121406.9253281
Cost s

KeyboardInterrupt: ignored

## Create the baseline RI_MAML_Agents_Coord

* Here, create RI_MAML_Agetnts, `agent_ind.py` in CityLearn
* week: Feb 15 ~ 19, I remove unnecessary parts from RI_Agents_Coord and save it as the first version of RI_MAML_Agents.
* week: Feb 22 ~ , before creating RI_MAML_Agents, I create RI_REINFORCE_Agents to checke my understanding.

In [12]:
import os
import torch.optim as optim
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions import Normal
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.utils as utils
import numpy as np
import random
import copy
import gym
from sklearn.linear_model import LinearRegression
import json
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
# torch.autograd.set_detect_anomaly(True)
import time
import math

# set the path so that we could read created modules
import sys

# organized into different classes for editing easier
from RBC_Agent import RBC_Agent
from model import PolicyNetwork, SoftQNetwork, RegressionBuffer
from ReplayBuffer import ReplayBuffer
from data_process import no_normalization, periodic_normalization, onehot_encoding, normalize, remove_feature

# check cuda is available
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
        

## Creating REINFORCE agents
* MARISA reward, REINFORCE agent for each building, PCA and information shareing are removed.
* *Reference*: [pytorch-REINFORCE](https://github.com/chingyaoc/pytorch-REINFORCE)

    ![image](https://user-images.githubusercontent.com/56372825/108765699-1d8c3d80-7522-11eb-9c26-508e71c9fe08.png)


#### Policy network class

In [13]:
class Policy(nn.Module):
    def __init__(self, hidden_size, num_inputs, action_space):
        super(Policy, self).__init__()
        self.action_space = action_space
        num_outputs = action_space.shape[0]

        self.linear1 = nn.Linear(num_inputs, hidden_size[0])
        self.linear1h = nn.Linear(hidden_size[0], hidden_size[1])
        self.linear2 = nn.Linear(hidden_size[1], num_outputs)
        self.linear2_ = nn.Linear(hidden_size[1], num_outputs)

    def forward(self, inputs):
        x = inputs
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear1h(x))
        mu = self.linear2(x)
        sigma_sq = self.linear2_(x)

        return mu, sigma_sq

In [14]:
pi = Variable(torch.FloatTensor([math.pi])).cuda()
def normal(x, mu, sigma_sq):
    a = (-1*(Variable(x)-mu).pow(2)/(2*sigma_sq)).exp()
    b = 1/(2*sigma_sq*pi.expand_as(sigma_sq)).sqrt()
    return a*b

#### RI_REINFORCE_Agents class

* REINFORCE updates Each policy for each buildings 
* Multiple buildings, MARISA reward, and no information sharing / PCA.

In [22]:
class RI_REINFORCE_Agents():
    def __init__(self, building_ids, 
                 buildings_states_actions, 
                 building_info,
                 observation_spaces = None, 
                 action_spaces = None):
        
        # Read valid state variables and action variables for each building
        # they are written in buildings_state_action_space.json in CityLean
        with open(buildings_states_actions) as json_file:
            self.buildings_states_actions = json.load(json_file)

        # Set seeds
        seed = 20180517 # to init parameters
        torch.manual_seed(seed)
        np.random.seed(seed)

        # Set action and ovservation spaces for each building
        # e.g., 'Building_1' as uid
        self.building_ids = building_ids 
        self.action_spaces = {uid : a_space for uid, a_space in zip(building_ids, action_spaces)}
        self.observation_spaces = {uid : o_space for uid, o_space in zip(building_ids, observation_spaces)}
    
        # Initialize a policy and optimizers
        # using empty dict, because keys, building_ids, are str.
        self.policies, self.optimizers = {}, {}
        for uid in self.building_ids:

            # initialize plicy parameters of a policy
            hidden_size = [400, 300]
            num_inputs = self.observation_spaces[uid].shape[0]

            # initialize a policy for a building
            self.policies[uid] = Policy(hidden_size, num_inputs, self.action_spaces[uid])
            self.policies[uid] = self.policies[uid].cuda()
            # initialize a optimizer for a policy
            self.optimizers[uid] = optim.Adam(self.policies[uid].parameters(), lr=1e-3)
            # set the plicy in training mode
            self.policies[uid].train()
         
    def select_action(self, states):
        # states is a ndarray, so first chage it as dict of uid
        states = {uid : o_space for uid, o_space in zip(self.building_ids, states)}
        # initialize actions as dict
        actions, log_probs, entropies = {}, {}, {}

        # for each policy
        for uid in self.building_ids:
            # get a state for a bilding and convert it to torch.float32
            state = torch.Tensor(states[uid])
            mu, sigma_sq = self.policies[uid](Variable(state).cuda())
            sigma_sq = F.softplus(sigma_sq)

            eps = torch.randn(mu.size())
            # calculate the probability
            action = (mu + sigma_sq.sqrt()*Variable(eps).cuda()).data
            
            prob = normal(action, mu, sigma_sq)

            entropy = -0.5*((sigma_sq+2*pi.expand_as(sigma_sq)).log()+1)
            log_prob = prob.log()
            
            actions[uid] = action.detach().cpu().numpy()
            entropies[uid] = entropy
            log_probs[uid] = log_prob

        # changes the type of actions from dict to list of np.ndarray
        return list(actions.values()), list(log_probs.values()), list(entropies.values())

    def update_policies(self, rewards, log_probs, entropies, gamma):
        # for each policy
        index = 0
        for uid in self.building_ids:
            # Dim of rewards is (number of steps) x (number of buildings)
            # So take a one reward sequence (step 0 ~ 8759) for one building using uid
            f = lambda x: x[index]
            reward_one_building = [f(x) for x in rewards]
            log_prob_one_building = [f(x) for x in log_probs]
            entropy_one_building = [f(x) for x in entropies]
            R = torch.zeros(1, )
            loss = 0
            for i in reversed(range(len(reward_one_building))):
                R = gamma * R + reward_one_building[i]
                loss = loss - (log_prob_one_building[i]*(Variable(R).expand_as(log_prob_one_building[i])).cuda()).sum() - (0.0001*entropy_one_building[i].cuda()).sum()
            loss = loss / len(reward_one_building)
		
            self.optimizers[uid].zero_grad()
            loss.backward()
            utils.clip_grad_norm(self.policies[uid].parameters(), 40)
            self.optimizers[uid].step()

            index += 1


#### Checking how the REINFORCE agents work correctly

In [23]:
agent = RI_REINFORCE_Agents(building_ids, 
                             building_state_actions, 
                             building_info, 
                             observations_spaces, 
                             actions_spaces)

In [24]:
n_episodes = 12
gamma = 0.99

costs = []
cumulated_rewards = []

for i_episode in range(n_episodes):
    print(f"Episode:{i_episode}")
    state = env.reset()
    done = False

    entropies = []
    log_probs = []
    rewards = []
    
    while not done:
        action, log_prob, entropy = agent.select_action(state)

        next_state, reward, done, _ = env.step(action)

        entropies.append(entropy)
        log_probs.append(log_prob)
        rewards.append(reward)
        state = next_state

        # every step, update the parameters
        # agent.update_policies(rewards, log_probs, entropies, gamma)
        # entropies = []
        # log_probs = []
        # rewards = []           
        
        if done:
            costs.append(env.cost())
            cumulated_rewards.append(np.sum(rewards))
            break
    # Here is the first position of update_policies
    agent.update_policies(rewards, log_probs, entropies, gamma)

Episode:0
Cumulated reward: -197272960.87538075
Cost score: {'ramping': 1.2480551, '1-load_factor': 1.1182843828045774, 'average_daily_peak': 1.1841508, 'peak_demand': 1.2459329, 'net_electricity_consumption': 1.049227, 'total': 1.1691300468413721}
Episode:1
Cumulated reward: -168369793.212318
Cost score: {'ramping': 1.0726278, '1-load_factor': 1.094463854584757, 'average_daily_peak': 1.0670604, 'peak_demand': 1.29738, 'net_electricity_consumption': 1.0330071, 'total': 1.1129078208513385}
Episode:2
Cumulated reward: -160121322.6677437
Cost score: {'ramping': 0.8927121, '1-load_factor': 1.0440851260273463, 'average_daily_peak': 1.0308814, 'peak_demand': 1.217092, 'net_electricity_consumption': 1.0353706, 'total': 1.04402825452981}
Episode:3
Cumulated reward: -159464957.52690515
Cost score: {'ramping': 0.929452, '1-load_factor': 1.0424108860147443, 'average_daily_peak': 1.0374389, 'peak_demand': 1.198293, 'net_electricity_consumption': 1.0362958, 'total': 1.048778099959144}
Episode:4
Cum